## Datashading 2010 US Census data with HoloViews/GeoViews

The regular census.ipynb notebook from the [datashader](https://github.com/bokeh/datashader) library shows a variety of analyses of the 2010 Census data on population and race.  Here we show how the latest 1.7 release of HoloViews (due by end of March 2017) will allow a much simpler syntax for instantiating interactive datashader plots, including support for layouts, overlays, etc.

### Set up and load data

In [ ]:
import pandas as pd
import holoviews as hv
import geoviews as gv
import datashader as ds

from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
hv.notebook_extension('bokeh')

%time df = pd.read_hdf('data/census.h5', 'census')
df.tail()

### Interactive plot of population density with race as colors, overlaid on a geographic map

This 300-million-point dataset is far too large to visualize in a browser directly as Points, but it's simple to wrap it in a `datashade` operation that aggregates the data into an Image before passing it to the browser.

In [ ]:
%%opts Overlay [width=900 height=525 xaxis=None yaxis=None] WMTS (alpha=0.5)

geomap = gv.WMTS(WMTSTileSource(url=\
   'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))

points = hv.Points(gv.Dataset(df, kdims=['meterswest', 'metersnorth'], vdims=['race']))

color_key = {'w':'blue', 'b':'green', 'a':'red', 'h':'orange', 'o':'saddlebrown'}
race = datashade(points, x_sampling=50, y_sampling=50, cmap=color_key,
                 element_type=gv.Image, aggregator=ds.count_cat('race'))

geomap * race